In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np


df = pd.read_csv('./data/df.csv',sep=',')
df  = df.sort_values(by='time',ascending=True).reset_index(drop=True)

BATCHSIZE = 256
LOOKBACK = 100


date = ["2024-02-19",
"2024-03-15",
"2024-04-19",
"2024-05-17",
"2024-06-21",
"2024-07-19",
"2024-08-16",
"2024-09-20",
"2024-10-18",
"2024-11-15",
"2024-12-20",
"2025-01-17"]
df.head()

df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df['date'] = df['date'].astype(str)
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute 
# for i in range(1,5):
#     df[f'spread_shift_{i}'] = df.groupby(['hour','minute'])['spread'].shift(i)

df['Expiration_Date'] = df['date'].apply(lambda x: 1 if x in date else 0)
# df.fillna(0,inplace=True)


df.dropna(inplace=True)

df.head()


,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date
0,2024-01-24 09:30:00,3240.0,3236.8,-3.2,2024-01-24,9,30,0
1,2024-01-24 09:31:00,3241.4,3243.0,1.6,2024-01-24,9,31,0
2,2024-01-24 09:32:00,3240.0,3243.0,3.0,2024-01-24,9,32,0
3,2024-01-24 09:33:00,3234.4,3235.6,1.2,2024-01-24,9,33,0
4,2024-01-24 09:34:00,3235.8,3236.0,0.2,2024-01-24,9,34,0


In [11]:
timeseries = df[["spread"]].values.astype('float32')

# plt.plot(timeseries[:,0])
# plt.show()

In [3]:
# train-test split for time series



import torch

def create_dataset(dataset, lookback,split_index):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """


    X_train, y_train = [], []
    X_test, y_test = [], []
    for i in range(len(dataset)-lookback):
        if i < split_index:
            feature = dataset[i:i+lookback]
            target = dataset[i+lookback:i+lookback+1][:,0:1]
            X_train.append(feature)
            y_train.append(target)
        else:
            feature = dataset[i:i+lookback]
            target = dataset[i+lookback:i+lookback+1][:,0:1]
            X_test.append(feature)
            y_test.append(target)
    # X = np.array(X)
    # y = np.array(y)
    return torch.tensor(X_train), torch.tensor(y_train),torch.tensor(X_test), torch.tensor(y_test)





In [4]:
import torch
import torch.nn as nn
import numpy as np

class CNN(nn.Module):
    def __init__(self, window_size=5):
        super().__init__()
        # 使用大窗口卷积捕获低频趋势
        self.trend_extractor = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=window_size, padding='same'),
            nn.ReLU(),
            nn.Conv1d(16, 1, kernel_size=window_size, padding='same')
        )
        
    def forward(self, x):
        # 输入形状: (batch_size, seq_len)
        x = x.permute(0,2,1)  # 添加通道维度 -> (B, 1, T)
        # print(x.shape)
        trend = self.trend_extractor(x)  # (B, 1, T)
        # residual = x - trend
        return trend

# # 使用示例
# model = DeepTrendDecomposer(window_size=7)
# input_seq = torch.randn(32, 100)  # batch_size=32, seq_len=100
# trend, residual = model(input_seq)


In [5]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, verbose=False):
        """
        Early stopping to stop the training when the loss does not improve after
        certain epochs.
        
        Args:
            patience (int): How many epochs to wait before stopping when loss is
                           not improving
            min_delta (float): Minimum change in the monitored quantity to
                             qualify as an improvement
            verbose (bool): If True, prints a message for each validation loss improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        
    def __call__(self, val_loss, model, optimizer, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, optimizer, epoch)
        elif val_loss > self.best_loss + self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, optimizer, epoch)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model, optimizer, epoch):
        """Saves model when validation loss decreases."""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [6]:
# train-test split for time series



import torch

def create_dataset(dataset, lookback,split_index):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """


    X_train, y_train = [], []
    X_test, y_test = [], []
    for i in range(len(dataset)-lookback):
        if i < split_index:
            feature = dataset[i:i+lookback]
            target = dataset[i+lookback:i+lookback+1][:,0:1]
            X_train.append(feature)
            y_train.append(target)
        else:
            feature = dataset[i:i+lookback]
            target = dataset[i+lookback:i+lookback+1][:,0:1]
            X_test.append(feature)
            y_test.append(target)
    # X = np.array(X)
    # y = np.array(y)
    return torch.tensor(X_train), torch.tensor(y_train),torch.tensor(X_test), torch.tensor(y_test)





In [13]:


class LSTMFCNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=50):
        super().__init__()
        # self.decomposer =  CNN(window_size=7)
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, 1)


    def forward(self, x):
        # 分解趋势
        # trend = self.decomposer(x)
        # print(trend.shape,residual.shape)
        x,_ = self.lstm(x)
        
        # 用LSTM预测残差
        # trend = trend.permute(0,2,1)
        # residual = residual.permute(0,2,1)
        # residual = residual.unsqueeze(-1)  # (B, T, 1)
        # lstm_out, _ = self.lstm(trend)
        x = self.linear(x)
        
        # 预测趋势（简单移动平均）
        # pred_trend = trend[:, -1].unsqueeze(-1)  # 假设趋势缓慢变化
        
        return x
# model = TrendAwareLSTM().to(device)
# model 

# sum(para.numel() for para in model.parameters())

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMFCNN().to(device)
model 

sum(para.numel() for para in model.parameters())

10651

In [15]:
device

device(type='cuda')

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data
import time 
begin_time = time.time()
df_result = pd.DataFrame()

for split_index in range(39237,len(timeseries),100): #, len(timeseries)
    train_size = split_index # 
    # test_size = len(timeseries) - train_size
    # train, test = timeseries[:train_size], timeseries[train_size:]
    lookback = LOOKBACK 
    X_train_all, y_train_all,X_test, y_test = create_dataset(timeseries, lookback=lookback,split_index=split_index)
    # X_test, y_test = create_dataset(test, lookback=lookback)
    print(X_train_all.shape, y_train_all.shape)
    print(X_test.shape, y_test.shape)

    indices = np.arange(len(X_train_all))   
    np.random.shuffle(indices)
    train_val_size = int(len(X_train_all) * 0.8)
    train_indices = indices[:train_val_size]
    val_indices = indices[train_val_size:]
    X_train = X_train_all[train_indices]
    y_train = y_train_all[train_indices]
    X_val = X_train_all[val_indices]
    y_val = y_train_all[val_indices]

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_val = X_val.to(device)
    y_val = y_val.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)

    model = LSTMFCNN().to(device)
    model 


    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.MSELoss()
    loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=BATCHSIZE)
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)
    n_epochs = 1000
    for epoch in range(n_epochs):
        model.train()
        for X_batch, y_batch in loader:
            # print(X_batch.shape)
            # X_batch = X_batch.to(device)
            # y_batch = y_batch.to(device)
            y_pred = model(X_batch)
            y_pred_val = model(X_val)
            # print(y_pred.shape,y_batch.shape)
            loss_train = loss_fn(y_pred, y_batch)
            
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
        # Validation
        loss_val = loss_fn(y_pred_val, y_val)
        early_stopping(loss_val, model, optimizer, epoch)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    model.eval()
    with torch.no_grad():
        # X_train = X_train.to(device)
        # y_train = y_train.to(device)
        y_pred_train = model(X_train)
        # X_test = X_test.to(device)
        # y_test = y_test.to(device)
        train_rmse = torch.sqrt(loss_fn(y_pred_train, y_train))
        y_pred_test = model(X_test)
        test_rmse = torch.sqrt(loss_fn(y_pred_test, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))
    end_time = time.time()
    print(f"训练时间: {end_time - begin_time} 秒")

    predict_value = model(X_test[0,:,:].unsqueeze(0))
    predict_value = predict_value.detach().cpu().numpy().flatten()


    df_result_tmp = pd.DataFrame([[split_index, predict_value[0],y_test[0].flatten().cpu().numpy()[0]]],columns=['split_index','predict','true'])
    df_result_tmp
    # df_result = pd.concat([df_result,df_result_tmp],ignore_index=True)
    df_result_tmp.to_csv(f'./data/lstm_fcnn_result_{LOOKBACK}.csv',index=False,mode='a')
    
    del model
    del optimizer
    del loader
    del early_stopping
    del X_train
    del y_train
    del X_train_all
    del y_train_all
    del X_val
    del y_val
    del X_test
    del y_test
    del y_pred_train
    del y_pred_test
    del y_pred_val
    del y_pred
    del y_batch
    del X_batch
    torch.cuda.empty_cache()
    

torch.Size([39237, 100, 1]) torch.Size([39237, 1, 1])
torch.Size([19226, 100, 1]) torch.Size([19226, 1, 1])


c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([256, 1, 1])) that is different to the input size (torch.Size([256, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([157, 1, 1])) that is different to the input size (torch.Size([157, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([7848, 1, 1])) that is different to the input size (torch.Size([7848, 100, 1])). This will likely lead to incorr

Early stopping


c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([31389, 1, 1])) that is different to the input size (torch.Size([31389, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([19226, 1, 1])) that is different to the input size (torch.Size([19226, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 118: train RMSE 2.2466, test RMSE 6.3871
训练时间: 151.14854454994202 秒
torch.Size([39337, 100, 1]) torch.Size([39337, 1, 1])
torch.Size([19126, 100, 1]) torch.Size([19126, 1, 1])


...

c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10568, 1, 1])) that is different to the input size (torch.Size([10568, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Early stopping


c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([42269, 1, 1])) that is different to the input size (torch.Size([42269, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([5626, 1, 1])) that is different to the input size (torch.Size([5626, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 98: train RMSE 3.2680, test RMSE 1.7039
训练时间: 32706.256201982498 秒
torch.Size([52937, 100, 1]) torch.Size([52937, 1, 1])
torch.Size([5526, 100, 1]) torch.Size([5526, 1, 1])


c:\Users\gaoxc\.conda\envs\chatglm3-demo\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10588, 1, 1])) that is different to the input size (torch.Size([10588, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
